In [1]:
import pandas as pd

In [ ]:
# open each result file
# try to parse json result
# check if columns are correct
# create a table with term type, term, normalized term

In [37]:
# Read JSON file
dir_name = 'mistral_results_section/mistral_results_section'

In [51]:
start_token = '<s> [INST] '
end_token = '[/INST]'
prompt = 'Extract all medical terms in a JSON format from the medical article below. The JSON result should include the type of medical term as "type", the term in the original language as "term" and the normalized English term as "normalized_term" from the text:'

In [52]:
import os

file_contents = []
for file in os.listdir(dir_name):
    filename = os.fsdecode(file)
    #filename = 'es-S0004-06142007000900010-10.txt'
    if filename.endswith(".txt"):
        with open(f'{dir_name}/{filename}') as file:
            text = file.read()            
            start_response = text.index(end_token)+len(end_token)
            
            is_complete = True
            if '</s>' in text[start_response:]:
                end_response = text[start_response:].index('</s>')
            else: 
                end_response = len(text[start_response:])-1
                is_complete = False
            
            start_request = len(start_token) + len(prompt)
            request = text[start_request: start_response-7].strip()
            response = text[start_response:start_response+end_response].strip()
            file_contents.append({
                'filename': filename,
                'request': request,
                'response': response,
                'is_complete': is_complete
            })
    else:
        continue

In [53]:
file_contents[0]

{'filename': 'es-S0004-06142007000900010-10.txt',
 'request': 'Se trata de un paciente de 59 años que ingresa con una herida de arma de fuego con orificio de entrada en la nalga, de unos 4 milímetros de diámetro y sin orificio de salida. Además presenta otra herida de bala en el muslo derecho con orificio de entrada por detrás y salida por delante, sin afectación de estructuras importantes. El paciente está consciente, orientado y con una ligera defensa abdominal. En la radiografía simple de abdomen aparece una bala situada a la altura de la sínfisis del pubis, así como una fractura de la rama descendente derecha del mismo. El enfermo llega en retención, siendo imposible el sondaje, por lo que se pone un catéter suprapúbico. Después se efectúa una laparotomía, no encontrándose ninguna otra lesión, realizándose una colostomía izquierda. Al día siguiente se realiza una uretrografía, comprobándose que hay paso de contraste desde la uretra hasta el recto, lo que provoca fiebre alta con esc

In [54]:
df_file_contents = pd.DataFrame.from_records(file_contents)
df_file_contents.head()

,filename,request,response,is_complete
0,es-S0004-06142007000900010-10.txt,Se trata de un paciente de 59 años que ingresa...,"[{""type"": ""Body Part"", ""term"": ""nalga"", ""norma...",True
1,es-S0004-06142007000900010-11427.txt,"Acto seguido, en el mismo acto quirúrgico, se ...","[{""type"": ""AnatomicalStructure"", ""term"": ""sínf...",True
2,es-S0004-06142007000900010-11987.txt,A los tres meses el paciente está asintomático...,"[\n {\n ""type"": ""Procedure"",\n ""term"": ...",True
3,es-S0004-06142007000900012-10.txt,Mujer de 67 años de edad con antecedentes pers...,"[{""type"": ""Disease"", ""term"": ""colecitonefritis...",True
4,es-S0004-06142007000900012-11507.txt,La paciente cursa tras la intervención sin inc...,"Based on the provided text, there are no expli...",True


In [55]:
df_file_contents.to_csv('mistral_file_contents_es.tsv', sep='\t', index=False)

In [57]:
df_file_contents[~df_file_contents['response'].str.startswith('[')]

,filename,request,response,is_complete
4,es-S0004-06142007000900012-11507.txt,La paciente cursa tras la intervención sin inc...,"Based on the provided text, there are no expli...",True
12,es-S0004-06142008000300012-14394.txt,ANATOMÍA PATOLÓGICA: angiomiolipoma de predomi...,"{\n\n ""terms"": [\n {\n ""type"": ""Disease"",\n...",True
48,es-S0210-48062005000700012-11889.txt,El paciente está asintomático con buen estado ...,"Based on the text provided, there are no medic...",True
79,es-S0211-57352013000300011-10.txt,Antecedentes y aparición del Delirio de Capgra...,"{\n""medical_terms"": [\n{\n""type"": ""Disease"",\n...",False
104,es-S0212-16112010000600024-14532.txt,"Dada la edad del paciente, la enfermedad actua...","Based on the given text, there are no explicit...",True
362,es-S1134-80462005000300004-10.txt,Presentamos el caso de un varón de 53 años de ...,"{\n""terms"": [\n{\n""type"": ""Disease"",\n""term"": ...",False
370,es-S1134-80462014000500004-1590.txt,- Intervención quirúrgica de exostosis a nivel...,"{\n""terms"": [\n{\n""type"": ""Condition"",\n""term""...",False


In [60]:
df_file_contents[~df_file_contents['response'].str.endswith(']')]

,filename,request,response,is_complete
4,es-S0004-06142007000900012-11507.txt,La paciente cursa tras la intervención sin inc...,"Based on the provided text, there are no expli...",True
9,es-S0004-06142008000300012-10.txt,Mujer de 41 años sin antecedentes de interés q...,"[{""type"": ""symptom"", ""term"": ""dolor abdominal ...",False
10,es-S0004-06142008000300012-11366.txt,Debido a este último dato del TC junto con la ...,"[{""type"": ""Organ"", ""term"": ""TC"", ""normalized_t...",False
12,es-S0004-06142008000300012-14394.txt,ANATOMÍA PATOLÓGICA: angiomiolipoma de predomi...,"{\n\n ""terms"": [\n {\n ""type"": ""Disease"",\n...",True
21,es-S0004-06142008000400016-10.txt,Presentamos el caso de un varón de 70 años con...,"[{""type"": ""disease"", ""term"": ""adenocarcinoma g...",True
...,...,...,...,...
482,es-S1889-836X2015000200005-20.txt,Describimos el caso de una paciente de 58 años...,"[\n {\n ""type"": ""Disease"",\n ""term"": ""m...",False
494,S0004-06142006000100014-1561.txt,Dados los antecedentes de adenocarcinoma gástr...,"[{""type"": ""Disease"", ""term"": ""adenocarcinoma g...",False
495,S0004-06142006000500002-20.txt,Paciente de 46 años que consultó por dolor a n...,"[\n{""type"": ""Disease"", ""term"": ""prostatitis"", ...",False
500,S0004-06142007000200017-10.txt,Paciente varón de 29 años. Es remitido a nuest...,"[\n{""type"": ""Body Part"", ""term"": ""testículo iz...",False


In [59]:
df_file_contents.iloc[12]['response']

'{\n\n "terms": [\n  {\n   "type": "Disease",\n   "term": "angiomiolipoma",\n   "normalized_term": "angiomiolipoma"\n  },\n  {\n   "type": "Disease",\n   "term": "de predominio lipomatoso",\n   "normalized_term": "lipomatous predominant"\n  },\n  {\n   "type": "Anatomy",\n   "term": "seno renal",\n   "normalized_term": "renal vein"\n  },\n  {\n   "type": "Anatomy",\n   "term": "anatomía patológica",\n   "normalized_term": "pathologic anatomy"\n  },\n  {\n   "type": "Anatomy",\n   "term": "infiltración vascular",\n   "normalized_term": "vascular infiltration"\n  }\n ]\n\n}'

In [62]:
import json
def parse_json(text):
    json_object = None
    try:
        json_object = json.loads(text)
    except ValueError as e:
        return None #rerun all not valid jsons
    return json_object

In [63]:
df_file_contents['json'] = df_file_contents['response'].apply(parse_json)

In [64]:
df_file_contents.head()

,filename,request,response,is_complete,json
0,es-S0004-06142007000900010-10.txt,Se trata de un paciente de 59 años que ingresa...,"[{""type"": ""Body Part"", ""term"": ""nalga"", ""norma...",True,"[{'type': 'Body Part', 'term': 'nalga', 'norma..."
1,es-S0004-06142007000900010-11427.txt,"Acto seguido, en el mismo acto quirúrgico, se ...","[{""type"": ""AnatomicalStructure"", ""term"": ""sínf...",True,"[{'type': 'AnatomicalStructure', 'term': 'sínf..."
2,es-S0004-06142007000900010-11987.txt,A los tres meses el paciente está asintomático...,"[\n {\n ""type"": ""Procedure"",\n ""term"": ...",True,"[{'type': 'Procedure', 'term': 'colonoscopia',..."
3,es-S0004-06142007000900012-10.txt,Mujer de 67 años de edad con antecedentes pers...,"[{""type"": ""Disease"", ""term"": ""colecitonefritis...",True,"[{'type': 'Disease', 'term': 'colecitonefritis..."
4,es-S0004-06142007000900012-11507.txt,La paciente cursa tras la intervención sin inc...,"Based on the provided text, there are no expli...",True,None


In [68]:
responses = []

for index, row in df_file_contents.iterrows():
    obj_type = type(row['json'])
    if obj_type is list:
        for item in row['json']:
            item['filename'] = row['filename']
            responses.append(item)
    elif obj_type is dict:
        for item in row['json']['terms']:
            item['filename'] = row['filename']
            responses.append(item)

In [69]:
df_response = pd.DataFrame.from_records(responses)

In [70]:
df_response.head()

,type,term,normalized_term,filename,value
0,Body Part,nalga,hip,es-S0004-06142007000900010-10.txt,NaN
1,Body Part,muslo derecho,right thigh,es-S0004-06142007000900010-10.txt,NaN
2,Body Part,sínfisis del pubis,pubic symphysis,es-S0004-06142007000900010-10.txt,NaN
3,Medical Procedure,sondaje,exploration,es-S0004-06142007000900010-10.txt,NaN
4,Medical Device,catéter suprapúbico,suprapubic catheter,es-S0004-06142007000900010-10.txt,NaN


In [80]:
import re
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return ' '.join([m.group(0) for m in matches])

In [81]:
def normalize_type(term_type):
    term_type = term_type.replace('_', ' ')
    term_type = camel_case_split(term_type)
    
    return term_type.lower()

In [82]:
normalize_type('MedicalProcedure')

'medical procedure'

In [83]:
df_response['normalized_type'] = df_response['type'].apply(normalize_type)

In [92]:
df_response[df_response['normalized_type']==df_response['normalized_term'].str.lower()]

,type,term,normalized_term,filename,value,normalized_type
447,Patient,paciente,patient,es-S0210-48062004000800009-10.txt,NaN,patient
1378,patient,paciente,patient,es-S0213-12852005000400003-10.txt,NaN,patient
1505,Hooks,ganchos,hooks,es-S0213-12852007000100002-12188.txt,NaN,hooks
1511,Prosthesis,protesis,prosthesis,es-S0213-12852007000100002-12576.txt,NaN,prosthesis
1512,Tissues,tejidos,tissues,es-S0213-12852007000100002-12576.txt,NaN,tissues
1513,Cicatrization,cicatrización,cicatrization,es-S0213-12852007000100002-12576.txt,NaN,cicatrization
1514,Conditions,condiciones,conditions,es-S0213-12852007000100002-12576.txt,NaN,conditions
1515,Implants,implantes,implants,es-S0213-12852007000100002-12576.txt,NaN,implants
1516,Teeth,dientes,teeth,es-S0213-12852007000100002-12576.txt,NaN,teeth
1705,Visual field,campo visual,visual field,es-S0365-66912007000100009-11242.txt,NaN,visual field


In [93]:
len(list(df_response['normalized_type'].unique()))

300

In [94]:
df_response.to_csv('mistral_es_mentions.tsv', sep='\t', index=False)

In [96]:
df_response['normalized_type'].value_counts().to_csv('mistral_es_mentions_type_counts.tsv', sep='\t')

In [ ]:
# find mapping in original text for each term

In [100]:
df_response[df_response['filename'] == 'es-S1698-69462006000100003-10.txt']

,type,term,normalized_term,filename,value,normalized_type
5469,Patient,Hombre,Man,es-S1698-69462006000100003-10.txt,NaN,patient
5470,Age,52 años,52 years,es-S1698-69462006000100003-10.txt,NaN,age
5471,Race,blanca,Caucasian,es-S1698-69462006000100003-10.txt,NaN,race
5472,Medical_Institution,Facultad de Odontología de la UDELAR de Montev...,"Faculty of Dentistry of UDELAR, Montevideo",es-S1698-69462006000100003-10.txt,NaN,medical institution
5473,Symptom,dolor,pain,es-S1698-69462006000100003-10.txt,NaN,symptom
5474,Location,una lesión exofítica en mucosa yugal derecha,an exophytic lesion on the right yugal mucosa,es-S1698-69462006000100003-10.txt,NaN,location
5475,Medical_History,ocasionales sangrados intestinales,occasional intestinal bleeding,es-S1698-69462006000100003-10.txt,NaN,medical history
5476,Family_Medical_History,padre fallecido por tumor intestinal,father died of intestinal tumor,es-S1698-69462006000100003-10.txt,NaN,family medical history
5477,Family_Medical_History,hermano operado de pólipos intestinales,brother operated on for intestinal polyps,es-S1698-69462006000100003-10.txt,NaN,family medical history
5478,Family_Medical_History,hermana histerectomizada recientemente con oof...,sister had recent hysterectomy with bilateral ...,es-S1698-69462006000100003-10.txt,NaN,family medical history


In [97]:
df_file_contents_idx = df_file_contents.set_index('filename')

In [98]:
df_file_contents_idx.loc['es-S1698-69462006000100003-10.txt']['request']

request        Hombre de 52 años, de raza blanca, que consult...
response       [{"type": "Patient", "term": "Hombre", "normal...
is_complete                                                 True
json           [{'type': 'Patient', 'term': 'Hombre', 'normal...
Name: es-S1698-69462006000100003-10.txt, dtype: object

In [106]:
def find_matches(mention, text):
    results = [] #mention, start, end
    matches = re.finditer(mention, text)
    return [(m.start(0), m.end(0)) for m in matches]

In [110]:
find_matches('paciente', df_file_contents_idx.loc['es-S1698-69462006000100003-10.txt']['request'])

[(515, 523)]

In [ ]:
# foreach find all matches
# find nonoverlapping
# save

In [111]:
all_matches = []
for index, row in df_file_contents.iterrows():
    text = row['request']
    mentions = df_response[df_response['filename'] == row['filename']]
    
    for midx, mrow in mentions.iterrows():
        matches = find_matches(mrow['term'], text)
        
        for match in matches:
            all_matches.append({
                'filename': row['filename'],
                'term': mrow['term'],
                'start': match[0],
                'end': match[1],
                'type': mrow['type'],
                'normalized_type': mrow['normalized_type'],
                'normalized_term': mrow['normalized_term']
            })

In [112]:
df_all_matches = pd.DataFrame.from_records(all_matches)
df_all_matches.head()

,filename,term,start,end,type,normalized_type,normalized_term
0,es-S0004-06142007000900010-10.txt,nalga,109,114,Body Part,body part,hip
1,es-S0004-06142007000900010-10.txt,muslo derecho,217,230,Body Part,body part,right thigh
2,es-S0004-06142007000900010-10.txt,sínfisis del pubis,485,503,Body Part,body part,pubic symphysis
3,es-S0004-06142007000900010-10.txt,sondaje,620,627,Medical Procedure,medical procedure,exploration
4,es-S0004-06142007000900010-10.txt,catéter suprapúbico,651,670,Medical Device,medical device,suprapubic catheter


In [126]:
df_all_matches.shape

(6337, 7)

In [114]:
df_all_matches.to_csv('all_matches.tsv', sep='\t', index=False)

In [ ]:
#transform from section to full file

In [139]:
df_sentences = pd.read_csv('test_file_sections.tsv', sep='\t')
df_sentences.head()

,filename,sentence_start,sentence_end,text
0,es-S0004-06142007000900010-1.txt,0,1425,Se trata de un paciente de 59 años que ingresa...
1,es-S0004-06142007000900010-1.txt,1427,1985,"Acto seguido, en el mismo acto quirúrgico, se ..."
2,es-S0004-06142007000900010-1.txt,1987,2567,A los tres meses el paciente está asintomático...
3,es-S0004-06142007000900012-1.txt,0,649,Mujer de 67 años de edad con antecedentes pers...
4,es-S0004-06142007000900012-1.txt,651,1505,En el estudio macroscópico se describe una neo...


In [140]:
df_sentences['section_name'] = df_sentences['filename'].apply(lambda x: str(x).replace('.txt', ''))

for index, row in df_sentences.iterrows():
    df_sentences.at[index, 'section_name'] = row['section_name'] + str(row['sentence_start']) + '.txt'
    
df_sentences.head()

,filename,sentence_start,sentence_end,text,section_name
0,es-S0004-06142007000900010-1.txt,0,1425,Se trata de un paciente de 59 años que ingresa...,es-S0004-06142007000900010-10.txt
1,es-S0004-06142007000900010-1.txt,1427,1985,"Acto seguido, en el mismo acto quirúrgico, se ...",es-S0004-06142007000900010-11427.txt
2,es-S0004-06142007000900010-1.txt,1987,2567,A los tres meses el paciente está asintomático...,es-S0004-06142007000900010-11987.txt
3,es-S0004-06142007000900012-1.txt,0,649,Mujer de 67 años de edad con antecedentes pers...,es-S0004-06142007000900012-10.txt
4,es-S0004-06142007000900012-1.txt,651,1505,En el estudio macroscópico se describe una neo...,es-S0004-06142007000900012-1651.txt


In [142]:
df_sentences_idx = df_sentences.set_index('section_name')

In [143]:
df_sentences_idx.loc['es-S0004-06142007000900010-11427.txt']

filename                           es-S0004-06142007000900010-1.txt
sentence_start                                                 1427
sentence_end                                                   1985
text              Acto seguido, en el mismo acto quirúrgico, se ...
Name: es-S0004-06142007000900010-11427.txt, dtype: object

In [144]:
df_all_matches['file_start'] = df_all_matches['start']
df_all_matches['file_end'] = df_all_matches['end']
df_all_matches['original_filename'] = df_all_matches['filename']

for index, row in df_all_matches.iterrows():
    original_filename_row = df_sentences_idx.loc[row['filename']]
        
    df_all_matches.at[index, 'original_filename'] = original_filename_row['filename']
    
    df_all_matches.at[index, 'file_start'] = original_filename_row['sentence_start'] + row['start']
    df_all_matches.at[index, 'file_end'] = original_filename_row['sentence_start'] + row['end']    
    
df_all_matches.head()

,filename,term,start,end,type,normalized_type,normalized_term,file_start,file_end,original_filename
0,es-S0004-06142007000900010-10.txt,nalga,109,114,Body Part,body part,hip,109,114,es-S0004-06142007000900010-1.txt
1,es-S0004-06142007000900010-10.txt,muslo derecho,217,230,Body Part,body part,right thigh,217,230,es-S0004-06142007000900010-1.txt
2,es-S0004-06142007000900010-10.txt,sínfisis del pubis,485,503,Body Part,body part,pubic symphysis,485,503,es-S0004-06142007000900010-1.txt
3,es-S0004-06142007000900010-10.txt,sondaje,620,627,Medical Procedure,medical procedure,exploration,620,627,es-S0004-06142007000900010-1.txt
4,es-S0004-06142007000900010-10.txt,catéter suprapúbico,651,670,Medical Device,medical device,suprapubic catheter,651,670,es-S0004-06142007000900010-1.txt


In [145]:
df_all_matches.head(50)

,filename,term,start,end,type,normalized_type,normalized_term,file_start,file_end,original_filename
0,es-S0004-06142007000900010-10.txt,nalga,109,114,Body Part,body part,hip,109,114,es-S0004-06142007000900010-1.txt
1,es-S0004-06142007000900010-10.txt,muslo derecho,217,230,Body Part,body part,right thigh,217,230,es-S0004-06142007000900010-1.txt
2,es-S0004-06142007000900010-10.txt,sínfisis del pubis,485,503,Body Part,body part,pubic symphysis,485,503,es-S0004-06142007000900010-1.txt
3,es-S0004-06142007000900010-10.txt,sondaje,620,627,Medical Procedure,medical procedure,exploration,620,627,es-S0004-06142007000900010-1.txt
4,es-S0004-06142007000900010-10.txt,catéter suprapúbico,651,670,Medical Device,medical device,suprapubic catheter,651,670,es-S0004-06142007000900010-1.txt
5,es-S0004-06142007000900010-10.txt,laparotomía,695,706,Medical Procedure,medical procedure,laparotomy,695,706,es-S0004-06142007000900010-1.txt
6,es-S0004-06142007000900010-10.txt,colostomía izquierda,763,783,Medical Procedure,medical procedure,left colostomy,763,783,es-S0004-06142007000900010-1.txt
7,es-S0004-06142007000900010-10.txt,uretrografía,817,829,Medical Procedure,medical procedure,urogram,817,829,es-S0004-06142007000900010-1.txt
8,es-S0004-06142007000900010-10.txt,uretra,880,886,Body Part,body part,urethra,880,886,es-S0004-06142007000900010-1.txt
9,es-S0004-06142007000900010-10.txt,uretra,1206,1212,Body Part,body part,urethra,1206,1212,es-S0004-06142007000900010-1.txt


In [147]:
current_filenames = list(df_all_matches['original_filename'].unique())

In [146]:
df_all_matches.to_csv('all_matches.tsv', sep='\t', index=False)

In [155]:
df_all_matches_eval = df_all_matches[['file_start', 'file_end', 'original_filename']]
df_all_matches_eval = df_all_matches_eval.rename(columns={'original_filename': 'note_id', 'file_start': 'start', 'file_end': 'end'})
df_all_matches_eval['note_id'] = df_all_matches_eval['note_id'].apply(lambda x: x.replace('.txt', ''))
df_all_matches_eval['concept_id']=1
df_all_matches_eval[['note_id','start','end','concept_id']].to_csv('pred_annotations_ner.csv', index=False)

In [148]:
#create test annotations - only for specific files

df_combined = pd.read_csv('combined_tsv_test_subtask1.tsv', sep='\t')
df_combined.head()

,filename,ann_id,label,start_span,end_span,text
0,es-S1138-123X2005000200006-2,T1,ENFERMEDAD,13,27,labio leporino
1,es-S1138-123X2005000200006-2,T2,ENFERMEDAD,30,55,fisura palatina bilateral
2,es-S1138-123X2005000200006-2,T3,ENFERMEDAD,60,95,premaxila estaba protruida y rotada
3,es-S0211-69952011000100019-1,T1,ENFERMEDAD,644,659,derrame pleural
4,es-S0211-69952011000100019-1,T2,ENFERMEDAD,45,64,glucogenosis tipo V


In [149]:
df_combined['is_in_list'] = df_combined['filename'].apply(lambda x: (x + '.txt') in current_filenames)

In [153]:
df_combined_in_list = df_combined[df_combined['is_in_list']]

In [154]:
df_combined_in_list = df_combined_in_list.rename(columns={'filename': 'note_id', 'start_span': 'start', 'end_span': 'end'})
df_combined_in_list['concept_id']=1
df_combined_in_list[['note_id','start','end','concept_id']].to_csv('test_annotations_ner.csv', index=False)